# 1. `Products_In_Sales_And_Purchase_Orders.sql`


## Proposition
Retrieve products that appear in both sales orders and purchase orders, focusing on products with significant sales and purchase quantities.

## Tables
- `Production.Product`: Stores information about products being produced, including `ProductID` and `Name`.
- `Sales.SalesOrderDetail`: Stores the details of products in each sales order, including `ProductID` and `OrderQty`.
- `Purchasing.PurchaseOrderDetail`: Contains details of products in each purchase order, such as `ProductID` and `Quantity`.

## Columns
- `ProductID`: The unique identifier for each product.
- `Name`: The name of the product.
- `OrderQty`: The quantity ordered in sales.
- `PurchaseQty`: The quantity ordered in purchasing.

In [ ]:
USE AdventureWorks2019

SELECT DISTINCT p.ProductID, p.Name, sod.OrderQty, pod.OrderQty AS PurchaseQty
FROM Production.Product p
JOIN Sales.SalesOrderDetail sod ON p.ProductID = sod.ProductID
JOIN Purchasing.PurchaseOrderDetail pod ON p.ProductID = pod.ProductID
WHERE sod.OrderQty > 10 AND pod.OrderQty > 5;

___
___
___

# 2. `All_Employees_Involved_In_Sales.sql`

## Proposition
Retrieve all employees who are involved in sales activities, either as salespersons or participants in sales transactions.

## Tables
- `HumanResources.Employee`: Stores information about employees, such as `EmployeeID`, `JobTitle`, and `HireDate`.
- `Sales.SalesOrderHeader`: Represents sales orders placed by customers, including details such as `SalesPersonID`, `OrderDate`, and `TotalDue`.

## Columns
- `EmployeeID`: The unique identifier for each employee.
- `JobTitle`: The title of the employee.
- `SalesOrderID`: The unique identifier for each sales order.

In [ ]:
USE AdventureWorks2019

SELECT DISTINCT e.BusinessEntityID, e.JobTitle, soh.SalesOrderID
FROM HumanResources.Employee e
LEFT JOIN Sales.SalesOrderHeader soh ON e.BusinessEntityID = soh.SalesPersonID
WHERE e.JobTitle LIKE '%Sales%' OR soh.SalesOrderID IS NOT NULL;

___
___
___

# 3. `Employees_Sales_Intersect.sql`

## Proposition
This query retrieves salespersons who have both achieved more than $500,000 in `SalesYTD`. It uses `INTERSECT` to find common records between these two conditions, displaying their `BusinessEntityID`, `FirstName`, `LastName`, and `SalesYTD`.

## Tables
- `Sales.SalesPerson`: Contains information about salespersons, including sales data and bonuses.
- `Person.Person`: Contains personal details of individuals, including employees.

## Columns
- `p.BusinessEntityID`: Unique identifier for each person from the `Person.Person` table.
- `p.FirstName`: The first name of the salesperson from the `Person.Person` table.
- `p.LastName`: The last name of the salesperson from the `Person.Person` table.
- `s.SalesYTD`: Total year-to-date sales for each salesperson from the `Sales.SalesPerson` table.


In [ ]:
USE AdventureWorks2019

SELECT p.BusinessEntityID, p.FirstName, p.LastName, s.SalesYTD
FROM Sales.SalesPerson s
JOIN Person.Person p ON s.BusinessEntityID = p.BusinessEntityID
WHERE s.SalesYTD > 500000

INTERSECT

SELECT p.BusinessEntityID, p.FirstName, p.LastName, s.SalesYTD
FROM Sales.SalesPerson s
JOIN Person.Person p ON s.BusinessEntityID = p.BusinessEntityID

___
___
___

# 4. `Recently_Purchased_Or_High_Reorder_Point.sql`

## Proposition
Retrieve products that are either in production, recently purchased in significant quantities, or have a high reorder point to identify products with active or ongoing demand.

## Tables
- `Production.Product`: Stores information about products being produced, such as `ProductID`, `Name`, `MakeFlag`, and `ReorderPoint`.
- `Purchasing.PurchaseOrderDetail`: Contains details about products in each purchase order, such as `ProductID` and `OrderQty`.

## Columns
- `ProductID`: The unique identifier for each product.
- `Name`: The name of the product.
- `MakeFlag`: Indicates if the product is currently in production.
- `OrderQty`: The quantity ordered in recent purchases.
- `ReorderPoint`: The level of inventory at which a reorder is triggered.

In [ ]:
USE AdventureWorks2019

SELECT DISTINCT p.ProductID, p.Name, p.MakeFlag, pod.OrderQty, p.ReorderPoint
FROM Production.Product p
LEFT JOIN Purchasing.PurchaseOrderDetail pod ON p.ProductID = pod.ProductID
WHERE p.MakeFlag = 1 OR pod.OrderQty > 10 OR p.ReorderPoint > 50
ORDER BY p.ProductID;

___
___
___

# 5. `Employees_Orders.sql`

## Proposition
This query retrieves a list of current managers and former employees. It combines information from both groups using a `UNION`. The query displays their `BusinessEntityID`, `JobTitle`, `FirstName`, `LastName`, and an indicator (`EmploymentStatus`) specifying whether they are a current or former employee.

## Tables
- `HumanResources.Employee`: Contains employee job-related information.
- `Person.Person`: Contains personal details of employees.
- `HumanResources.EmployeeDepartmentHistory`: Contains historical department assignment information for employees.

## Columns
- `e.BusinessEntityID`: Unique identifier for each employee from the `HumanResources.Employee` table.
- `e.JobTitle`: Title of the employee, used to filter for managers.
- `p.FirstName`: The first name of the employee from the `Person.Person` table.
- `p.LastName`: The last name of the employee from the `Person.Person` table.
- `EmploymentStatus`: Derived field to indicate if an employee is currently employed or a former employee.


In [ ]:
USE AdventureWorks2019

SELECT e.BusinessEntityID, e.JobTitle, p.FirstName, p.LastName, 'Current Employee' AS EmploymentStatus
FROM HumanResources.Employee e
JOIN Person.Person p ON e.BusinessEntityID = p.BusinessEntityID
WHERE e.JobTitle LIKE '%Manager%'

UNION

SELECT e.BusinessEntityID, e.JobTitle, p.FirstName, p.LastName, 'Former Employee' AS EmploymentStatus
FROM HumanResources.Employee e
JOIN Person.Person p ON e.BusinessEntityID = p.BusinessEntityID
WHERE e.BusinessEntityID NOT IN (SELECT BusinessEntityID FROM HumanResources.EmployeeDepartmentHistory WHERE EndDate IS NULL);


___
___
___

# 6. `Customers_Sales.sql`

## Proposition
This query retrieves a list of customers who have either a high-sales-related account number or have been recently modified. It uses `UNION` to combine both groups, displaying their `CustomerID`, `AccountNumber`, `FirstName`, `LastName`, and an indicator (`CustomerType`) specifying whether they are a high-sales customer or a recent customer.

## Tables
- `Sales.Customer`: Contains information about customer accounts.
- `Person.Person`: Contains personal details of customers.

## Columns
- `c.CustomerID`: Unique identifier for each customer from the `Sales.Customer` table.
- `c.AccountNumber`: The account number of the customer from the `Sales.Customer` table.
- `p.FirstName`: The first name of the customer from the `Person.Person` table.
- `p.LastName`: The last name of the customer from the `Person.Person` table.
- `CustomerType`: Derived field to indicate if the customer is a high-sales customer or a recent customer.


In [ ]:
USE AdventureWorks2019

SELECT c.CustomerID, c.AccountNumber, p.FirstName, p.LastName, 'High Sales Customer' AS CustomerType
FROM Sales.Customer c
JOIN Person.Person p ON c.PersonID = p.BusinessEntityID
WHERE c.AccountNumber LIKE 'AW%'

UNION

SELECT c.CustomerID, c.AccountNumber, p.FirstName, p.LastName, 'Preferred Customer' AS CustomerType
FROM Sales.Customer c
JOIN Person.Person p ON c.PersonID = p.BusinessEntityID
WHERE c.AccountNumber LIKE 'AW%' AND c.TerritoryID IN (1, 2, 3);

___
___
___

# 7. `Preferred_Customers.sql`

## Proposition
This query retrieves a list of customers who either have high sales and recent high-value orders, or are classified as preferred customers. It uses a combination of `UNION` and `INTERSECT` to combine these groups, displaying their `CustomerID`, `AccountNumber`, `FirstName`, `LastName`, and a customer type indicator (`CustomerType`).

## Tables
- `Sales.Customer`: Contains information about customer accounts.
- `Person.Person`: Contains personal details of customers.
- `Sales.SalesOrderHeader`: Contains information about sales orders, including total order amounts.

## Columns
- `CustomerID`: Unique identifier for each customer.
- `AccountNumber`: The account number of the customer.
- `FirstName`: The first name of the customer.
- `LastName`: The last name of the customer.
- `CustomerType`: Derived field to indicate if the customer is a high sales customer with recent high-value orders, or a preferred customer.

## Common Table Expressions (CTEs)
- `HighSales`: Contains customers with high sales accounts (`AccountNumber` starting with 'AW%').
- `PreferredCustomers`: Contains customers with high sales accounts and belonging to specific territories (TerritoryID 1, 2, or 3).
- `RecentHighValueOrders`: Contains customers who have placed recent orders with a total value greater than $10,000.

In [ ]:
USE AdventureWorks2019

;WITH HighSales AS (
    SELECT c.CustomerID, c.AccountNumber, p.FirstName, p.LastName
    FROM Sales.Customer c
    JOIN Person.Person p ON c.PersonID = p.BusinessEntityID
    WHERE c.AccountNumber LIKE 'AW%'
),
PreferredCustomers AS (
    SELECT c.CustomerID, c.AccountNumber, p.FirstName, p.LastName
    FROM Sales.Customer c
    JOIN Person.Person p ON c.PersonID = p.BusinessEntityID
    WHERE c.AccountNumber LIKE 'AW%' AND c.TerritoryID IN (1, 2, 3)
),
RecentHighValueOrders AS (
    SELECT o.CustomerID
    FROM Sales.SalesOrderHeader o
    WHERE o.TotalDue > 10000
)

SELECT hs.CustomerID, hs.AccountNumber, hs.FirstName, hs.LastName, 'High Sales and Recent High Value Order' AS CustomerType
FROM HighSales hs
INTERSECT
SELECT rhvo.CustomerID, hs.AccountNumber, hs.FirstName, hs.LastName, 'High Sales and Recent High Value Order' AS CustomerType
FROM RecentHighValueOrders rhvo
JOIN HighSales hs ON rhvo.CustomerID = hs.CustomerID

UNION

SELECT pc.CustomerID, pc.AccountNumber, pc.FirstName, pc.LastName, 'Preferred Customer' AS CustomerType
FROM PreferredCustomers pc;


___
___
___

# 8. `High_Value_Frequent_Customers.sql`

## Proposition
This query retrieves a list of customers classified into different categories such as loyal customers, high-value preferred customers, and frequent high-value customers. It uses a combination of `UNION` and `INTERSECT` to combine these groups, displaying their `CustomerID`, `AccountNumber`, `FirstName`, `LastName`, and a customer type indicator (`CustomerType`).

## Tables
- `Sales.Customer`: Contains information about customer accounts.
- `Person.Person`: Contains personal details of customers.
- `Sales.SalesOrderHeader`: Contains information about sales orders, including order dates and total values.

## Columns
- `CustomerID`: Unique identifier for each customer.
- `AccountNumber`: The account number of the customer.
- `FirstName`: The first name of the customer.
- `LastName`: The last name of the customer.
- `CustomerType`: Derived field to indicate if the customer is a loyal customer, a high-value preferred customer, or a frequent high-value customer.

## Common Table Expressions (CTEs)
- `LoyalCustomers`: Contains customers who have high sales accounts (`AccountNumber` starting with 'AW%') and have placed more than 5 orders.
- `HighValuePreferred`: Contains customers who have high sales accounts, belong to specific territories (TerritoryID 1, 2, or 3), and have orders with a total value greater than $15,000.
- `FrequentRecentOrders`: Contains customers who have placed more than 3 orders within the last year.

In [ ]:
USE AdventureWorks2019

;WITH LoyalCustomers AS (
    SELECT c.CustomerID, c.AccountNumber, p.FirstName, p.LastName
    FROM Sales.Customer c
    JOIN Person.Person p ON c.PersonID = p.BusinessEntityID
    WHERE c.AccountNumber LIKE 'AW%' AND c.CustomerID IN (
        SELECT o.CustomerID
        FROM Sales.SalesOrderHeader o
        GROUP BY o.CustomerID
        HAVING COUNT(o.SalesOrderID) > 5
    )
),
HighValuePreferred AS (
    SELECT c.CustomerID, c.AccountNumber, p.FirstName, p.LastName
    FROM Sales.Customer c
    JOIN Person.Person p ON c.PersonID = p.BusinessEntityID
    JOIN Sales.SalesOrderHeader o ON c.CustomerID = o.CustomerID
    WHERE c.AccountNumber LIKE 'AW%' AND c.TerritoryID IN (1, 2, 3) AND o.TotalDue > 15000
),
FrequentRecentOrders AS (
    SELECT o.CustomerID
    FROM Sales.SalesOrderHeader o
    WHERE o.OrderDate >= DATEADD(year, -1, GETDATE())
    GROUP BY o.CustomerID
    HAVING COUNT(o.SalesOrderID) > 3
)

SELECT lc.CustomerID, lc.AccountNumber, lc.FirstName, lc.LastName, 'Loyal Customer' AS CustomerType
FROM LoyalCustomers lc

UNION

SELECT hv.CustomerID, hv.AccountNumber, hv.FirstName, hv.LastName, 'High Value Preferred Customer' AS CustomerType
FROM HighValuePreferred hv

INTERSECT

SELECT fr.CustomerID, hv.AccountNumber, hv.FirstName, hv.LastName, 'Frequent High Value Customer' AS CustomerType
FROM FrequentRecentOrders fr
JOIN HighValuePreferred hv ON fr.CustomerID = hv.CustomerID;

___
___
___

# 9. `High_Value_Recent_Spenders.sql`

## Proposition
This query retrieves a list of customers classified into different categories such as frequent high-value customers, recent high spenders, and high-value customers. It uses a combination of `UNION` and `INTERSECT` to combine these groups, displaying their `CustomerID`, `AccountNumber`, `FirstName`, `LastName`, and a customer type indicator (`CustomerType`).

## Tables
- `Sales.SalesOrderHeader`: Contains information about sales orders, including order dates and total values.
- `Person.Person`: Contains personal details of customers.

## Columns
- `CustomerID`: Unique identifier for each customer.
- `AccountNumber`: The account number of the customer.
- `FirstName`: The first name of the customer.
- `LastName`: The last name of the customer.
- `CustomerType`: Derived field to indicate if the customer is a frequent high-value customer, a recent high spender, or a high-value customer.

## Common Table Expressions (CTEs)
- `RecentOrders`: Contains customers who have placed more than 2 orders within the last 3 months.
- `HighValueOrders`: Contains customers who have spent more than $30,000 in total on their orders.
- `FrequentHighValueCustomers`: Contains customers who meet both criteria for recent orders and high value, found using `INTERSECT`.


In [ ]:
USE AdventureWorks2019

;WITH RecentOrders AS (
    SELECT o.CustomerID, COUNT(o.SalesOrderID) AS OrderCount
    FROM Sales.SalesOrderHeader o
    WHERE o.OrderDate >= DATEADD(month, -3, GETDATE())
    GROUP BY o.CustomerID
    HAVING COUNT(o.SalesOrderID) > 2
),
HighValueOrders AS (
    SELECT o.CustomerID, SUM(o.TotalDue) AS TotalSpent
    FROM Sales.SalesOrderHeader o
    GROUP BY o.CustomerID
    HAVING SUM(o.TotalDue) > 10000
),
FrequentHighValueCustomers AS (
    SELECT ro.CustomerID
    FROM RecentOrders ro
    INTERSECT
    SELECT hvo.CustomerID
    FROM HighValueOrders hvo
)

SELECT p.BusinessEntityID, p.FirstName, p.LastName, 'Frequent High Value Customer' AS CustomerType
FROM Person.Person p
JOIN FrequentHighValueCustomers fhv ON p.BusinessEntityID = fhv.CustomerID

UNION

SELECT p.BusinessEntityID, p.FirstName, p.LastName, 'Recent High Spender' AS CustomerType
FROM Person.Person p
JOIN RecentOrders ro ON p.BusinessEntityID = ro.CustomerID
WHERE ro.OrderCount > 4

UNION

SELECT p.BusinessEntityID, p.FirstName, p.LastName, 'High Value Customer' AS CustomerType
FROM Person.Person p
JOIN HighValueOrders hvo ON p.BusinessEntityID = hvo.CustomerID;

___
___
___

# 10. `Products_And_Orders_With_Discounts.sql`

## Proposition
This query retrieves information about products and orders that have discounts applied. It uses a common table expression (CTE) to extract products with discounts and orders with discounts, then combines the results using a `UNION` statement.

## Tables
- `Production.Product`: Contains details of products.
- `Sales.SpecialOfferProduct`: Associates special offers with products.
- `Sales.SalesOrderDetail`: Contains details of products in each sales order.
- `Sales.SalesOrderHeader`: Contains general information about each sales order, including the order date.

## Columns
- `ProductDiscounts.ProductID`: Unique identifier for each product that has a discount applied.
- `ProductDiscounts.ProductName`: The name of the product.
- `ProductDiscounts.SpecialOfferID`: The identifier of the special offer applied to the product.
- `OrderDiscounts.SalesOrderID`: Unique identifier for each sales order that includes a product with a discount.
- `OrderDiscounts.OrderDate`: The date when the order was placed.
- `OrderDiscounts.SpecialOfferID`: The identifier of the special offer associated with the product in the order.


In [74]:
USE AdventureWorks2019

;WITH ProductDiscounts AS (
    SELECT DISTINCT p.ProductID, p.Name AS ProductName, sp.SpecialOfferID
    FROM Production.Product p
    LEFT JOIN Sales.SpecialOfferProduct sp ON p.ProductID = sp.ProductID
    WHERE sp.SpecialOfferID IS NOT NULL
),
OrderDiscounts AS (
    SELECT DISTINCT so.SalesOrderID, soh.OrderDate, so.SpecialOfferID
    FROM Sales.SalesOrderDetail so
    INNER JOIN Sales.SpecialOfferProduct sp ON so.ProductID = sp.ProductID
    INNER JOIN Sales.SalesOrderHeader soh ON so.SalesOrderID = soh.SalesOrderID
    WHERE sp.SpecialOfferID IS NOT NULL
)

SELECT ProductID, ProductName, SpecialOfferID
FROM ProductDiscounts
UNION
SELECT SalesOrderID, OrderDate, SpecialOfferID
FROM OrderDiscounts;

Total execution time: 00:00:00.018

ProductID,ProductName,SpecialOfferID


: Msg 241, Level 16, State 1, Line 3
Conversion failed when converting date and/or time from character string.